In [78]:
import autograd.numpy as anp
import autograd.numpy as anp
import pymanopt
import pymanopt.manifolds
import pymanopt.optimizers

import autograd.numpy as np
import time

def RTRNewton(X,y,z,gamma):
    time1 = time.time()
    (m,n) = X.shape
    Y = np.concatenate([0.5 * (gamma**0.5) * X, 0.5 * (gamma**0.5) * np.ones((m,1))], axis = 1)
    Lhat = np.concatenate([Y, 0.5 * z], axis = 1)
    #Lend = 0.5 * z - y
    #A = np.dot(Lhat.T, Lhat)
    #b = np.dot(Lhat.T, Lend)
    #c = np.dot(Lend.T, Lend)
    
    dim = n + 2
    manifold = pymanopt.manifolds.Sphere(dim)

    @pymanopt.function.autograd(manifold)
    def cost(r):
        return np.linalg.norm((Lhat @ r).reshape(y.shape[0],1) - y + z/2) ** 2

    problem = pymanopt.Problem(manifold, cost)

    optimizer = pymanopt.optimizers.trust_regions.TrustRegions()
    result = optimizer.run(problem)

    #print("Pymanopt solution:", result.point)
    time2 = time.time()
    return time2-time1

In [79]:
import random
#import numpy as np
def getTime_realData(X, y, z, gamma,size_range):
    fold = 10
    num_size = len(size_range)
    random.seed(2021)
    (m,n) = X.shape
    ssdp_yes = 1
    time_ssdp = np.zeros(num_size)
    time_ssdp_std = np.zeros(num_size)
    optval_ssdp = np.zeros(num_size)
    w_ssdp = np.zeros((n+1,fold))
    for i in range(num_size):
        print('Starting a dataset size of ' + str(i));
        timessdp = np.zeros(fold)
        optvalssdp = np.zeros(fold)
        m_curr = size_range[i]
        for idx in range(fold):
            
            ridx = random.sample(range(m), int(m_curr))
            if ssdp_yes:
                print('Doing ssdp')
                
                timessdp[idx] = RTRNewton(X[ridx] , y[ridx].reshape((-1,1)), z[ridx].reshape((-1,1)), gamma);  
        time_ssdp[i] = np.log(np.mean(timessdp))
        time_ssdp_std[i]= np.std(timessdp)
    return time_ssdp, time_ssdp_std

            
       
           
    
    

In [80]:
import os
#import numpy as np
import pandas as pd
from sympy import subsets


# Read data from database provided
# "name" as the name of the dataset
def data_read(name: str, directory="datasets/"):
    if name in ["wine_modest", "wine_severe"]:
        data = pd.read_csv(os.path.join(directory, "wine.csv"),
                           sep=';').to_numpy()
        X, y = data[:, :-1], data[:, -1]
        z = np.clip(y, 0, 10)
        if name.find("modest") != -1:
            z = np.clip(z, 6, None)
        else:
            z = np.clip(z, 7, None)
        # Configuration parameters, can change later using
        # structure options
        const = 0.1
        gamma_list = np.linspace(1e-3, 0.75, 40)
        gamma_time = 0.5
        datasize_list = np.linspace(150, 1500, 16)

    elif name in ["insurance_modest", "insurance_severe"]:
        data = pd.read_csv(os.path.join(directory,
                                        "insurance.csv"), sep=',')
       
        y = data["charges"].to_numpy()
        data = data.drop(columns=['charges'])
        
        for column in ["sex", "smoker", "region"]:
            one_hot = pd.get_dummies(data[column])
            data = data.drop(column, axis=1)
            data = data.join(one_hot)
        X = data.to_numpy()
        if name.find("modest"):
            z = y - 100
        else:
            z = y - 300
        z = np.clip(z, 0, None)
        z /= 100
        y /= 100

        # Configuration parameters
        const = 0.03
        gamma_list = [1e-3]
        gamma_time = 0.5
        datasize_list = np.linspace(100, 1300, 25)

    elif name in ["building_modest", "building_severe",
                  "building_modest170", "building_severe170"]:
        data = pd.read_excel(io=os.path.join(directory, "building.xlsx"),
                             sheet_name="Data", header=1).to_numpy()
        y = data[:, -1]
        X = data[:, :-1]
        if name.find("modest"):
            z = y + 20
        else:
            z = y + 40
        z = np.clip(z, 0, None)

        # Configuration parameters
        const = 0.01
        gamma_list = [20]
        gamma_time = 0.5
        if name in ["building_modest", "building_severe"]:
            datasize_list = np.linspace(100, 300, 21)
        else:
            datasize_list = [170]
        nor = const * np.max(np.abs(y))
        y = y / nor
        z = z / nor

    elif name in ["blog_modest", "blog_severe"]:
        data = pd.read_csv(os.path.join(directory, "blog.csv"),
                           sep=',').to_numpy()
        y = data[:, -1]
        X = data[:, :-1]
        if name.find("modest"):
            z = y + 5
        else:
            z = y + 10
        z = np.clip(z, 0, None)

        # Configuration parameters
        const = 0.01
        gamma_list = [200]
        gamma_time = 0.5
        datasize_list = np.linspace(5000, 50000, 10)

    else:
        raise ValueError("Data set name not recognized.")

    return X, y, z, const, gamma_list, gamma_time, datasize_list

In [81]:
from matplotlib.pyplot import table
import pandas as pd
from sklearn.preprocessing import normalize
dataname_list = ["wine_modest", "wine_severe", "insurance_modest","insurance_severe"]
len_name = len(dataname_list)
gamma_list = [1e-1]
normalize_yes = 1

for gamma_idx in range(len(gamma_list)):
    gamma = gamma_list[gamma_idx]
    ssdp_mr = np.zeros((len_name,1))
    ssdp_std = np.zeros((len_name,1))
    for idx in range(len_name):
        dataname = dataname_list[idx]
        [X, y, z, const, gamma_list, gamma_time, datasize_list] = data_read(name = dataname)
        
        if normalize_yes:
            X = normalize(X, axis=1, norm='l1')
        
        
        ssdp_mr, ssdp_std = getTime_realData(X, y, z, gamma, datasize_list)
        
        
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        print(ssdp_mr, ssdp_std)
        df = pd.DataFrame({'datasize': datasize_list, 'ssdp_mr': ssdp_mr, 'ssdp_std': ssdp_std})
         
        table_name_cmp ='result/'+str(dataname)+'_time.csv'
        df.to_csv(table_name_cmp)
        


   

Starting a dataset size of 0
Doing ssdp
Optimizing...
                                            f: +4.052537e+02   |grad|: 1.310567e+03
acc TR+   k:     1     num_inner:     0     f: +1.064354e+02   |grad|: 3.566737e+02   exceeded trust region
acc       k:     2     num_inner:     1     f: +6.909492e+01   |grad|: 7.281573e+01   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +6.663503e+01   |grad|: 8.305723e+00   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +6.647466e+01   |grad|: 1.249952e+01   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +6.625918e+01   |grad|: 6.070903e+00   reached target residual-kappa (linear)
REJ TR-   k:     6     num_inner:     1     f: +6.625918e+01   |grad|: 6.070903e+00   reached target residual-kappa (linear)
REJ TR-   k:     7     num_inner:     1     f: +6.625918e+01   |grad|: 6.070903e+00   exceeded trust region
acc TR+   k:     8   

Optimizing...
                                            f: +1.158115e+03   |grad|: 2.508241e+03
acc TR+   k:     1     num_inner:     0     f: +4.264977e+02   |grad|: 1.382480e+03   exceeded trust region
acc       k:     2     num_inner:     1     f: +8.546470e+01   |grad|: 3.023851e+02   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +5.730287e+01   |grad|: 5.795561e+01   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +5.572087e+01   |grad|: 5.694728e+00   reached target residual-kappa (linear)
REJ TR-   k:     5     num_inner:     1     f: +5.572087e+01   |grad|: 5.694728e+00   reached target residual-kappa (linear)
REJ TR-   k:     6     num_inner:     1     f: +5.572087e+01   |grad|: 5.694728e+00   exceeded trust region
acc TR+   k:     7     num_inner:     1     f: +5.563135e+01   |grad|: 1.963757e+00   exceeded trust region
acc TR+   k:     8     num_inner:     1     f: +5.549361e+01   |grad|: 4.79953

Optimizing...
                                            f: +3.093432e+03   |grad|: 5.017571e+03
acc TR+   k:     1     num_inner:     0     f: +1.462261e+03   |grad|: 3.560615e+03   exceeded trust region
acc       k:     2     num_inner:     1     f: +1.650200e+02   |grad|: 6.864601e+02   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +8.322591e+01   |grad|: 1.474529e+02   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +7.705929e+01   |grad|: 1.945005e+01   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +7.685634e+01   |grad|: 3.284240e+00   reached target residual-kappa (linear)
REJ TR-   k:     6     num_inner:     1     f: +7.685634e+01   |grad|: 3.284240e+00   reached target residual-kappa (linear)
REJ TR-   k:     7     num_inner:     1     f: +7.685634e+01   |grad|: 3.284240e+00   exceeded trust region
acc TR+   k:     8     num_inner:     1     f: +7.678709e+01 

acc       k:    15     num_inner:     1     f: +1.647759e+02   |grad|: 1.946367e+01   reached target residual-kappa (linear)
acc       k:    16     num_inner:     1     f: +1.645812e+02   |grad|: 8.418997e+00   reached target residual-kappa (linear)
acc       k:    17     num_inner:     1     f: +1.644884e+02   |grad|: 2.119067e+01   reached target residual-kappa (linear)
acc       k:    18     num_inner:     1     f: +1.643597e+02   |grad|: 2.815779e+00   reached target residual-kappa (linear)
REJ TR-   k:    19     num_inner:     1     f: +1.643597e+02   |grad|: 2.815779e+00   reached target residual-kappa (linear)
acc TR+   k:    20     num_inner:     1     f: +1.643156e+02   |grad|: 4.198273e+00   exceeded trust region
acc       k:    21     num_inner:     1     f: +1.642853e+02   |grad|: 8.276766e+00   reached target residual-kappa (linear)
acc       k:    22     num_inner:     1     f: +1.642641e+02   |grad|: 1.024112e+00   reached target residual-kappa (linear)
acc       k:    2

acc       k:    18     num_inner:     1     f: +2.079407e+02   |grad|: 2.136869e+01   exceeded trust region
acc       k:    19     num_inner:     1     f: +2.078406e+02   |grad|: 2.195993e+00   reached target residual-kappa (linear)
REJ TR-   k:    20     num_inner:     1     f: +2.078406e+02   |grad|: 2.195993e+00   exceeded trust region
acc TR+   k:    21     num_inner:     1     f: +2.078202e+02   |grad|: 1.268599e+00   exceeded trust region
acc TR+   k:    22     num_inner:     1     f: +2.077940e+02   |grad|: 5.055735e+00   exceeded trust region
acc       k:    23     num_inner:     1     f: +2.077766e+02   |grad|: 3.336646e+00   reached target residual-kappa (linear)
acc       k:    24     num_inner:     1     f: +2.077693e+02   |grad|: 1.762180e+00   reached target residual-kappa (linear)
acc       k:    25     num_inner:     1     f: +2.077673e+02   |grad|: 6.786527e-01   reached target residual-kappa (linear)
acc       k:    26     num_inner:     1     f: +2.077670e+02   |grad

...

acc       k:     3     num_inner:     1     f: +2.919365e+04   |grad|: 3.145989e+05   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +7.209734e+03   |grad|: 9.595709e+04   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +2.469772e+03   |grad|: 3.009861e+04   reached target residual-kappa (linear)
acc       k:     6     num_inner:     1     f: +1.372598e+03   |grad|: 9.230811e+03   reached target residual-kappa (linear)
acc       k:     7     num_inner:     1     f: +1.159548e+03   |grad|: 1.770562e+03   reached target residual-kappa (linear)
acc       k:     8     num_inner:     1     f: +1.149373e+03   |grad|: 2.769863e+01   reached target residual-kappa (linear)
acc       k:     9     num_inner:     1     f: +1.149371e+03   |grad|: 4.159459e-04   reached target residual-kappa (linear)
acc       k:    10     num_inner:     1     f: +1.149371e+03   |grad|: 5.821318e-11   reached target residual-theta (superlin

acc       k:     4     num_inner:     1     f: +1.046295e+04   |grad|: 1.335826e+05   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +3.193825e+03   |grad|: 4.156531e+04   reached target residual-kappa (linear)
acc       k:     6     num_inner:     1     f: +1.538868e+03   |grad|: 1.300435e+04   reached target residual-kappa (linear)
acc       k:     7     num_inner:     1     f: +1.181049e+03   |grad|: 3.156091e+03   reached target residual-kappa (linear)
acc       k:     8     num_inner:     1     f: +1.149440e+03   |grad|: 1.490905e+02   reached target residual-kappa (linear)
acc       k:     9     num_inner:     1     f: +1.149366e+03   |grad|: 3.420883e-02   reached target residual-kappa (linear)
acc       k:    10     num_inner:     1     f: +1.149366e+03   |grad|: 1.359643e-09   reached target residual-theta (superlinear)
Terminated - min grad norm reached after 10 iterations, 0.06 seconds.

Doing ssdp
Optimizing...
                        

acc       k:     3     num_inner:     1     f: +1.212299e+05   |grad|: 9.567302e+05   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +2.569622e+04   |grad|: 2.852242e+05   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +6.524545e+03   |grad|: 8.729305e+04   reached target residual-kappa (linear)
acc       k:     6     num_inner:     1     f: +2.348906e+03   |grad|: 2.745320e+04   reached target residual-kappa (linear)
acc       k:     7     num_inner:     1     f: +1.379883e+03   |grad|: 8.278910e+03   reached target residual-kappa (linear)
acc       k:     8     num_inner:     1     f: +1.205236e+03   |grad|: 1.372062e+03   reached target residual-kappa (linear)
acc       k:     9     num_inner:     1     f: +1.199343e+03   |grad|: 1.174634e+01   reached target residual-kappa (linear)
acc       k:    10     num_inner:     1     f: +1.199343e+03   |grad|: 6.962991e-05   reached target residual-kappa (linear)


acc       k:     2     num_inner:     1     f: +1.652369e+05   |grad|: 1.217014e+06   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +3.427113e+04   |grad|: 3.612119e+05   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +8.363611e+03   |grad|: 1.100174e+05   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +2.802284e+03   |grad|: 3.446366e+04   reached target residual-kappa (linear)
acc       k:     6     num_inner:     1     f: +1.518052e+03   |grad|: 1.061789e+04   reached target residual-kappa (linear)
acc       k:     7     num_inner:     1     f: +1.263017e+03   |grad|: 2.135792e+03   reached target residual-kappa (linear)
acc       k:     8     num_inner:     1     f: +1.249315e+03   |grad|: 4.148807e+01   reached target residual-kappa (linear)
acc       k:     9     num_inner:     1     f: +1.249310e+03   |grad|: 9.701854e-04   reached target residual-kappa (linear)


acc       k:    11     num_inner:     1     f: +1.299281e+03   |grad|: 5.959067e-12   reached target residual-theta (superlinear)
Terminated - min grad norm reached after 11 iterations, 0.03 seconds.

Doing ssdp
Optimizing...
                                            f: +1.059894e+07   |grad|: 2.110124e+07
acc TR+   k:     1     num_inner:     0     f: +4.292619e+06   |grad|: 1.248109e+07   exceeded trust region
acc       k:     2     num_inner:     1     f: +6.146098e+05   |grad|: 3.262720e+06   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +1.161581e+05   |grad|: 9.455166e+05   reached target residual-kappa (linear)
acc       k:     4     num_inner:     1     f: +2.484910e+04   |grad|: 2.823830e+05   reached target residual-kappa (linear)
acc       k:     5     num_inner:     1     f: +6.421030e+03   |grad|: 8.656297e+04   reached target residual-kappa (linear)
acc       k:     6     num_inner:     1     f: +2.389702e+03   |grad|: 2.721068e+0